# Implement the pipeline for Named Entity Recognition with NLP

In [16]:
# Import libraries
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"This notebook was last compiled at: {datetime.datetime.now():%Y-%m-%d %H:%M:%S}")

This notebook was last compiled at: 2025-09-05 20:48:02


## Import dataset

In [ ]:
def load_data(data_dir, mode=None):
    df = pd.read_csv(data_dir, encoding="ISO-8859-1")
    return df

True


In [19]:
source_dir = "/mnt/e/Development/Python/NLP/NaturalLanguageProcessing/"
data_dir = source_dir + "data/archive/ner_dataset.csv"
print(os.path.exists(data_dir))
df = load_data(data_dir)

print(df.head(10))

True
    Sentence #           Word  POS    Tag
0  Sentence: 1      Thousands  NNS      O
1          NaN             of   IN      O
2          NaN  demonstrators  NNS      O
3          NaN           have  VBP      O
4          NaN        marched  VBN      O
5          NaN        through   IN      O
6          NaN         London  NNP  B-geo
7          NaN             to   TO      O
8          NaN        protest   VB      O
9          NaN            the   DT      O


## Preprocessing data

In [21]:
print(df.isna().sum())

Sentence #    1000616
Word               10
POS                 0
Tag                 0
dtype: int64


In [26]:
def preprocessing_data(data):
    # Fill NaN values in "Sentence #"
    data["Sentence #"] = data["Sentence #"].ffill()
    data["Sentence #"] = data["Sentence #"].astype(str)

    # Drop rows with missing Word
    data = data.dropna(subset=["Word"]).reset_index(drop=True)

    # Strip whitespace
    for col in ["Word", "POS", "Tag"]:
        data[col] = data[col].str.strip()
        
    return data


In [27]:
df_prep = preprocessing_data(df)

In [29]:
# Groupby follow sentence for training
grouped = df.groupby("Sentence #").apply(
    lambda s: list(zip(s["Word"], s["Tag"]))
).tolist()

print(grouped[:10])


[[('Thousands', 'O'), ('of', 'O'), ('demonstrators', 'O'), ('have', 'O'), ('marched', 'O'), ('through', 'O'), ('London', 'B-geo'), ('to', 'O'), ('protest', 'O'), ('the', 'O'), ('war', 'O'), ('in', 'O'), ('Iraq', 'B-geo'), ('and', 'O'), ('demand', 'O'), ('the', 'O'), ('withdrawal', 'O'), ('of', 'O'), ('British', 'B-gpe'), ('troops', 'O'), ('from', 'O'), ('that', 'O'), ('country', 'O'), ('.', 'O')], [('Iranian', 'B-gpe'), ('officials', 'O'), ('say', 'O'), ('they', 'O'), ('expect', 'O'), ('to', 'O'), ('get', 'O'), ('access', 'O'), ('to', 'O'), ('sealed', 'O'), ('sensitive', 'O'), ('parts', 'O'), ('of', 'O'), ('the', 'O'), ('plant', 'O'), ('Wednesday', 'B-tim'), (',', 'O'), ('after', 'O'), ('an', 'O'), ('IAEA', 'B-org'), ('surveillance', 'O'), ('system', 'O'), ('begins', 'O'), ('functioning', 'O'), ('.', 'O')], [('Helicopter', 'O'), ('gunships', 'O'), ('Saturday', 'B-tim'), ('pounded', 'O'), ('militant', 'O'), ('hideouts', 'O'), ('in', 'O'), ('the', 'O'), ('Orakzai', 'B-geo'), ('tribal', '

/tmp/ipykernel_2568/755275269.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = df.groupby("Sentence #").apply(


## Encoding data